In [1]:
# Cell 1: Cài đặt các thư viện cần thiết
# Uncomment dòng dưới nếu chưa cài đặt transformers

# Cell 2: Import các thư viện
import torch
from transformers import T5ForSequenceClassification, AutoTokenizer
import os
import numpy as np

# Cell 3: Tải tokenizer và model
# Đường dẫn tới thư mục chứa các file model (.bin, .model, config.json, etc.)
model_dir = "."  # Thay đổi nếu cần

# Tải tokenizer từ local files
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Tải model T5ForSequenceClassification từ file pytorch_model.bin
model = T5ForSequenceClassification.from_pretrained(
    model_dir,
    local_files_only=True
)
model.eval()  # Chuyển sang chế độ evaluation

# Cell 4: Hàm dự đoán
def predict_toxicity(text, tokenizer, model):
    # Với mô hình T5, đôi khi cần prefix
    # Nếu model đã được fine-tune với một prefix cụ thể, hãy sử dụng nó
    # Ví dụ: prefix = "phân loại: "
    # text_with_prefix = prefix + text
    
    # Tokenize văn bản đầu vào
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Thực hiện dự đoán
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predictions = torch.argmax(probabilities, dim=-1)
    
    # Lấy kết quả và xác suất
    prediction = predictions.item()
    probability = probabilities[0][prediction].item()
    
    # Map label index to label name (0: non-toxic, 1: toxic)
    # Lưu ý: Bạn cần xác nhận thứ tự này dựa trên cách model được train
    label_map = {0: "non-toxic", 1: "toxic"}
    predicted_label = label_map[prediction]
    
    return {
        "label": predicted_label,
        "probability": probability,
        "toxic_score": probabilities[0][1].item() if probabilities.shape[1] > 1 else probability
    }

# Cell 5: Kiểm tra với một số ví dụ
# Thêm cell này để kiểm tra xem model load thành công chưa trước khi dự đoán
print(f"Model đã được tải: {model.__class__.__name__}")
print(f"Tokenizer đã được tải: {tokenizer.__class__.__name__}")

# Cell 6: Kiểm tra với một số ví dụ
test_texts = [
    "Chúc bạn một ngày tốt lành!",
    "Đồ ngu dốt, không biết gì cả.",
    "Buổi thuyết trình hôm nay thật tuyệt vời.",
    "Mày là đồ vô học, cút đi!",
    "Cái nhà này rất tệ",
]

for text in test_texts:
    result = predict_toxicity(text, tokenizer, model)
    print(f"Văn bản: '{text}'")
    print(f"Kết quả: {result['label']} (Điểm toxic: {result['toxic_score']:.4f})")
    print("-" * 50)

# Cell 7: Hàm để sử dụng model trong ứng dụng thực tế
def detect_toxic_content(text):
    """
    Hàm tiện ích để sử dụng trong ứng dụng
    
    Args:
        text (str): Văn bản tiếng Việt cần kiểm tra
        
    Returns:
        dict: Kết quả phân loại với nhãn và điểm số
    """
    return predict_toxicity(text, tokenizer, model)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Model đã được tải: T5ForSequenceClassification
Tokenizer đã được tải: T5TokenizerFast
Văn bản: 'Chúc bạn một ngày tốt lành!'
Kết quả: non-toxic (Điểm toxic: 0.0001)
--------------------------------------------------
Văn bản: 'Đồ ngu dốt, không biết gì cả.'
Kết quả: toxic (Điểm toxic: 0.9957)
--------------------------------------------------
Văn bản: 'Buổi thuyết trình hôm nay thật tuyệt vời.'
Kết quả: non-toxic (Điểm toxic: 0.0016)
--------------------------------------------------
Văn bản: 'Mày là đồ vô học, cút đi!'
Kết quả: toxic (Điểm toxic: 0.9761)
--------------------------------------------------
Văn bản: 'Cái nhà này rất tệ'
Kết quả: toxic (Điểm toxic: 0.6490)
--------------------------------------------------


In [2]:
# Cell 6: Kiểm tra với một số ví dụ, bổ sung in token IDs và kiểm tra độ dài
for text in test_texts:
    # Tokenize câu, trả về token IDs
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    token_ids = inputs["input_ids"][0].tolist()  # Lấy token ID dạng list
    
    # Kiểm tra độ dài token IDs
    length = len(token_ids)
    exceeds_max_len = length > 512
    
    print(f"Văn bản: '{text}'")
    print(f"Token IDs: {token_ids}")
    print(f"Độ dài token: {length} (Vượt 512?: {'Có' if exceeds_max_len else 'Không'})")
    
    # Dự đoán toxicity
    result = predict_toxicity(text, tokenizer, model)
    print(f"Kết quả: {result['label']} (Điểm toxic: {result['toxic_score']:.4f})")
    print("-" * 50)


Văn bản: 'Chúc bạn một ngày tốt lành!'
Token IDs: [8128, 1113, 68, 198, 1387, 3481, 35993, 1]
Độ dài token: 8 (Vượt 512?: Không)
Kết quả: non-toxic (Điểm toxic: 0.0001)
--------------------------------------------------
Văn bản: 'Đồ ngu dốt, không biết gì cả.'
Token IDs: [1287, 5784, 25975, 35790, 129, 300, 1080, 251, 35792, 1]
Độ dài token: 10 (Vượt 512?: Không)
Kết quả: toxic (Điểm toxic: 0.9957)
--------------------------------------------------
Văn bản: 'Buổi thuyết trình hôm nay thật tuyệt vời.'
Token IDs: [5856, 1736, 498, 1045, 714, 1655, 2498, 5137, 35792, 1]
Độ dài token: 10 (Vượt 512?: Không)
Kết quả: non-toxic (Điểm toxic: 0.0016)
--------------------------------------------------
Văn bản: 'Mày là đồ vô học, cút đi!'
Token IDs: [10297, 49, 1333, 1244, 268, 35790, 15241, 352, 35993, 1]
Độ dài token: 10 (Vượt 512?: Không)
Kết quả: toxic (Điểm toxic: 0.9761)
--------------------------------------------------
Văn bản: 'Cái nhà này rất tệ'
Token IDs: [2797, 233, 118, 726, 2235, 1